# Make Accent Trial Pairs

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#### Sample seed files
sample 10% seed files in a stratified manner according to accent label

In [2]:
test_files = pd.read_csv('../aishell_2_partitions/class_test.txt', sep="\t", header=None)
test_files.columns = ["file", "label"]
test_files['label2'] = test_files['label']
y = test_files.pop('label2')
X = test_files
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42, stratify=y)
seed_files = pd.concat([X_train, y_train], axis=1)
seed_files = seed_files.reset_index(drop=True)
seed_files.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5598 entries, 0 to 5597
Data columns (total 3 columns):
file      5598 non-null object
label     5598 non-null object
label2    5598 non-null object
dtypes: object(3)
memory usage: 131.3+ KB


In [3]:
for index, row in seed_files.head().iterrows():
    print(index, row[0], row[1])

0 ../aishell_2/data/wav/C0246/IC0246W0455.wav South
1 ../aishell_2/data/wav/C9068/IC9068W0362.wav South
2 ../aishell_2/data/wav/C0311/IC0311W0191.wav South
3 ../aishell_2/data/wav/D0185/ID0185W0018.wav North
4 ../aishell_2/data/wav/C9124/IC9124W0197.wav South


#### Generate trial pairs
Generate k trial pairs (from test pool) for each seed file, ensure uniform distribution for label matches. Let k = 8.

In [4]:
k = 8
with open('./aishell2_acc_test.txt', 'w') as outfile:
    for index, seed_row in seed_files.iterrows():
        for i in range(k):
            label = "none"
            if i % 2 == 0: # generate match pair
                match_label = 1
                while label != seed_row[1]:
                    idx, row = next(test_files.sample(n=1).iterrows())
                    label = row[1]
                outfile.write("{} {} {}\n".format(match_label, seed_row[0], row[0]))
            else: # generate mismatch pair
                match_label = 0
                while label == seed_row[1] or label == "none":
                    idx, row = next(test_files.sample(n=1).iterrows())
                    label = row[1]
                outfile.write("{} {} {}\n".format(match_label, seed_row[0], row[0]))